Project Goals:

Extract Place Entities (Cities, Locations, etc) from any document(novel, news articles, etc) and plot them on a Map.

User Flow:

- Enter document(pdf, text file, doc, etc) 
- Extract Locations
- Find Long and Lat values for location entities extracted 
- Create WordCloud of Locations
- Plot those coordinates on a map

Authors:
Nono Umasy, 
Jason Hamada



In [24]:
import pandas as pd 
import folium
import string
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from folium.plugins import MarkerCluster
from matplotlib.pyplot import figure



from nltk.corpus import stopwords
from wordcloud import WordCloud
from nltk import word_tokenize

In [11]:
geo_cities = "/Users/nonoumasy/Downloads/data/world-cities.csv"
geo_df = pd.read_csv(geo_cities)
geo_df = geo_df[['name','country']]

In [12]:
known_locations = set(geo_df['name'])
#known_locations |= set(geo_df['name'])

In [4]:
#get data
from collections import Counter

#stop words
stop_words = set(stopwords.words('english'))

#punctuation
punct_table = str.maketrans('', '', string.punctuation)
def punct_strip(text):
    return text.translate(punct_table)

def get_words(text):
    tokens = word_tokenize(text)
    stripped = (punct_strip(t) for t in tokens)
    words = (w for w in stripped if not w in stop_words)
    return words

loc_counter = Counter()

#with open('/Users/nonoumasy/Downloads/data/tolstoy.txt', 'rt') as f:
with open(input(), 'rt') as f:
    lines = (f.readline() for _ in range(100000))
    for line in lines:
        words = get_words(line)
        loc_counter.update(w for w in words if w in known_locations)

/Users/nonoumasy/Downloads/data/sherlock.txt


In [14]:
#get lontitude and latitude for cities
geolocator = Nominatim(user_agent="xray", timeout=5)
def lat_lon(city):
    loc = geolocator.geocode(city)
    if loc is None:
        raise AttributeError(f'city not found -- {city}')
    return {'city': city, 
                'lat': loc.latitude,
                'lon': loc.longitude}


coordinates = []
for city in loc_counter:
    try:
        coordinates.append(lat_lon(city))
    except AttributeError:
        pass
    
loc_df = pd.DataFrame(coordinates)

In [27]:
# plot cities on map

m = folium.Map(
    tiles = "Stamen Toner",
    location=[45, 37],
    zoom_start=4
)


marker_cluster = MarkerCluster().add_to(m)

loc_df.apply(lambda row:folium.CircleMarker(location=[row["lat"], row["lon"]], 
    radius=10,
    fill=True,
    color='#40B8AF',
    popup=city,
    ).add_to(marker_cluster), axis=1)

m.save('tolstoy.html')
m

## TODO

  * measure frequency of each location/character
  * display locations of each location/character
  * activate map cluster
  * tooltip/popup for each location
  * parameterize for different source text
  * improve NER
  * dash for website

In [0]:
#make wordcloud of cities